In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [ ]:
import pickle
import os
def save_obj(obj,path,name):
    with open(os.path.join(path, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [ ]:
!ls /content/drive/MyDrive/DDI/test_features/

bert_1		      BioBert_embedding.ipynb	  scibert_1
bert_2		      data_text_features.pkl	  scibert_2
bert_embedding.ipynb  ddi_test_id_w_features.pkl  scibert_embedding.ipynb
biobert_1	      id_list.pkl		  test_id_w_ground_truth.pkl
biobert_2	      name_list.pkl


In [ ]:
data_text_features = load_obj("/content/drive/MyDrive/DDI/test_features/data_text_features.pkl")
id_list = load_obj("/content/drive/MyDrive/DDI/test_features/id_list.pkl")
id_list = [int(_) for _ in id_list]
test_id = load_obj("/content/drive/MyDrive/DDI/test_features/test_id_w_ground_truth.pkl")

In [ ]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [ ]:
def mean_pooling_2(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, [0,1])
    sum_mask = torch.clamp(input_mask_expanded.sum([0,1]), min=1e-9)
    return sum_embeddings.unsqueeze(0) / sum_mask.unsqueeze(0)

In [ ]:
#Sentences we want sentence embeddings for
sentences = ['This framework generates embeddings for each input sentence',
             'Sentences are passed as a list of string.',
             'The quick brown fox jumps over the lazy dog when it is raining.']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

In [ ]:
#Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=256, return_tensors='pt')

#Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

#Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

len(data_text_features)

data = data_text_features[id_list.index(test_id[8])]

data[6]
striped_list = [_.strip() for _ in data[6]]
full_sent = ' '.join(striped_list)

len(full_sent.split())

  sl = 0
  i = 8
  bert_embedding=[]
  bert_embedding_2 = []
  if i == 8:
      data_list = []
      data_list_2 = []
      data = data_text_features[id_list.index(test_id[i])]
      print(i)
      
      for j in range(2,len(data)):
          if j == 4:
              print(j)
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              
              if len(full_sent.split()) > 512:
                  encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1].mean(0,keepdim=True))
              else:
                  encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1])
              continue
          elif j ==5:
              print(j)
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              if len(full_sent.split()) > 512:
                  encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1].mean(0,keepdim=True))
              else:
                  encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1])
              continue
          elif j == 6:
              print(j)
              striped_list = [_.strip() for _ in data[j]]
              full_sent = ' '.join(striped_list)
              if len(full_sent.split()) > 2000:
                  strip_index = 0
                  strip_count = 0
                  for si in range(len(striped_list)):
                      strip_count += len(striped_list[si].split())
                      if strip_count < 2000:
                          strip_index = si 
                      else:
                          break
              if len(striped_list) == 0: continue
              encoded_input = tokenizer(striped_list[:strip_index], padding=True, truncation=True, max_length=512, return_tensors='pt')
              #Compute token embeddings
              with torch.no_grad():
                  model_output = model(**encoded_input)
              #Perform pooling. In this case, mean pooling
              sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
              data_list.append(sentence_embeddings)
              data_list_2.append(model_output[1].mean(0,keepdim=True))
              continue
          else:
              print(j)
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
              #Compute token embeddings
              with torch.no_grad():
                  model_output = model(**encoded_input)
              #Perform pooling. In this case, mean pooling
              sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
              data_list.append(sentence_embeddings)
              data_list_2.append(model_output[1])
              continue
      bert_embedding.append(np.array(torch.cat(data_list,0)))
      bert_embedding_2.append(np.array(torch.cat(data_list_2,0)))
  bert_np = np.array(bert_embedding)
  bert_np_2 = np.array(bert_embedding_2)
  savedex = sl//100
  np.save("/content/drive/MyDrive/DDI/test_features/scibert_1/"+str(savedex),bert_np)
  np.save("/content/drive/MyDrive/DDI/test_features/scibert_2/"+str(savedex),bert_np_2)

In [ ]:
for sl in range(0,len(test_id),100):
  bert_embedding=[]
  bert_embedding_2 = []
  for i in range(sl,min(sl+100,len(test_id))):
      data_list = []
      data_list_2 = []
      data = data_text_features[id_list.index(test_id[i])]
      print(i)
      
      for j in range(2,len(data)):
          if j == 4:

              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              
              if len(full_sent.split()) > 512:
                  if len(full_sent.split()) > 1000:
                      strip_index = 0
                      strip_count = 0
                      for si in range(len(striped_list)):
                          strip_count += len(striped_list[si].split())
                          if strip_count < 1000:
                              strip_index = si 
                          else:
                              break
                      encoded_input = tokenizer(striped_list[:strip_index], padding=True, truncation=True, max_length=512, return_tensors='pt')
                  else:
                      encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1].mean(0,keepdim=True))
              else:
                  encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1])
              continue
          elif j ==5:
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)

              if len(full_sent.split()) > 512:
                  if len(full_sent.split()) > 1000:
                      strip_index = 0
                      strip_count = 0
                      for si in range(len(striped_list)):
                          strip_count += len(striped_list[si].split())
                          if strip_count < 1000:
                              strip_index = si 
                          else:
                              break
                      encoded_input = tokenizer(striped_list[:strip_index], padding=True, truncation=True, max_length=512, return_tensors='pt')
                  else:
                      encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1].mean(0,keepdim=True))
              else:
                  encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
                  #Compute token embeddings
                  with torch.no_grad():
                      model_output = model(**encoded_input)
                  #Perform pooling. In this case, mean pooling
                  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
                  data_list.append(sentence_embeddings)
                  data_list_2.append(model_output[1])
              continue
          elif j == 6:
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              if len(full_sent.split()) > 1000:
                  strip_index = 0
                  strip_count = 0
                  for si in range(len(striped_list)):
                      strip_count += len(striped_list[si].split())
                      if strip_count < 1000:
                          strip_index = si 
                      else:
                          break
                  encoded_input = tokenizer(striped_list[:strip_index], padding=True, truncation=True, max_length=512, return_tensors='pt')
              else:
                  encoded_input = tokenizer(striped_list, padding=True, truncation=True, max_length=512, return_tensors='pt')
              #Compute token embeddings
              with torch.no_grad():
                  model_output = model(**encoded_input)
              #Perform pooling. In this case, mean pooling
              sentence_embeddings = mean_pooling_2(model_output, encoded_input['attention_mask'])
              data_list.append(sentence_embeddings)
              data_list_2.append(model_output[1].mean(0,keepdim=True))
              continue
          else:
              striped_list = [_.strip() for _ in data[j]]
              if len(striped_list) == 0: continue
              full_sent = ' '.join(striped_list)
              encoded_input = tokenizer(full_sent, padding=True, truncation=True, max_length=512, return_tensors='pt')
              #Compute token embeddings
              with torch.no_grad():
                  model_output = model(**encoded_input)
              #Perform pooling. In this case, mean pooling
              sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
              data_list.append(sentence_embeddings)
              data_list_2.append(model_output[1])
              continue
      bert_embedding.append(np.array(torch.cat(data_list,0)))
      bert_embedding_2.append(np.array(torch.cat(data_list_2,0)))
  bert_np = np.array(bert_embedding)
  bert_np_2 = np.array(bert_embedding_2)
  savedex = sl//100
  np.save("/content/drive/MyDrive/DDI/test_features/scibert_1/"+str(savedex),bert_np)
  np.save("/content/drive/MyDrive/DDI/test_features/scibert_2/"+str(savedex),bert_np_2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

IndexError: ignored

8

In [ ]:
len(bert_embedding)

10

In [ ]:
  bert_np = np.array(bert_embedding)
  bert_np_2 = np.array(bert_embedding_2)
  savedex = sl//100
  np.save("/content/drive/MyDrive/DDI/test_features/scibert_1/"+str(savedex),bert_np)
  np.save("/content/drive/MyDrive/DDI/test_features/scibert_2/"+str(savedex),bert_np_2)

In [ ]:
bert_embedding[8][-1,:]

array([ 2.85345651e-02,  5.15232235e-02, -4.43149328e-01,  2.13090718e-01,
       -9.25425440e-02, -2.99583495e-01,  3.35589916e-01,  3.53259265e-01,
       -4.38257456e-02,  1.26213774e-01,  4.89209861e-01,  1.06283084e-01,
       -4.88520354e-01, -4.09429014e-01, -4.15095717e-01, -4.27250922e-01,
       -6.22350499e-02, -8.13323408e-02,  4.04027134e-01, -2.74545878e-01,
        1.62152514e-01,  3.63494277e-01,  1.07574677e-02, -3.74525458e-01,
        4.20936912e-01,  6.33021832e-01, -2.13137254e-01, -1.72041170e-02,
       -2.65098453e-01,  2.25656211e-01, -2.66928375e-01, -1.46909043e-01,
       -3.95662427e-01, -7.28853405e-01,  2.35857889e-01,  3.17745835e-01,
        2.77955800e-01, -9.66640413e-02, -4.06908244e-01,  1.37640480e-02,
       -2.53298700e-01, -1.82540491e-01,  1.00070572e+00, -1.54397085e-01,
        1.43043548e-01, -1.76235139e-01, -1.25615835e-01,  4.36090052e-01,
        1.76823258e-01, -1.20980203e-01, -3.17457155e-03, -1.51602495e-02,
        8.32283720e-02,  

In [ ]:
bert_np = np.array(bert_embedding)
bert_np_2 = np.array(bert_embedding_2)

In [ ]:
np.save("/content/drive/MyDrive/DDI/training/SciBert/0",bert_np[:-2,:,:])
np.save("/content/drive/MyDrive/DDI/training/SciBert_2/0",bert_np_2[:-2,:,:])

In [ ]:
bert_np.shape

(352, 5, 768)